**Sommario**

  - [Template parziali: l'struzione Jinja `include`](#template-parziali-lstruzione-jinja-include)
      - [L'istruzione `include`](#listruzione-include)
      - [Come usare `include` in Flask](#come-usare-include-in-flask)
      - [Perché usare `{% include ... %}`](#perch%C3%A9-usare-include)
  - [URL, *route* ed *endpoint* in Flask](#url-route-ed-endpoint-in-flask)
  - [Come leggere i parameri passati con GET e con POST](#come-leggere-i-parameri-passati-con-get-e-con-post)
      - [Richiesta GET](#richiesta-get)
    - [Richiesta POST](#richiesta-post)
      - [POST con dati di tipo FormData](#post-con-dati-di-tipo-formdata)
      - [POST con dati di tipo JSON](#post-con-dati-di-tipo-json)
    - [Riassumendo](#riassumendo)
  - [GET e POST: vantaggi e svantaggi](#get-e-post-vantaggi-e-svantaggi)
  - [`if __name__ == '__main__':`](#if-__name__-__main__)
  - [APPROFONDIMENTO Python: `range()` al contrario](#approfondimento-python-range-al-contrario)

## Template parziali: l'struzione Jinja `include`

Il motore di templating Jinja, consente di combinare assieme più template.

Ci sono due approcci principali:

- l'utilizzo di "sub-template" o "template parziali", il cui scopo è quello di essere inclusi in altri template "principali". Per fare questo si usa l'istruzione `{% include 'template_parziale.html' %}` all'interno del template principale.

- l'utilizzo dell'ereditarietà tra template, che consente di creare un template base con sezioni modificabili. Questo si ottiene definendo blocchi con l'istruzione `{% block nome_blocco %}{% endblock %}` nel template base e utilizzando `{% extends 'template_base.html' %}` nei template figli. I template figli possono sovrascrivere questi blocchi per personalizzare il contenuto specifico della pagina, mantenendo la struttura generale fornita dal template base.

Inizialmente concentriamoci sul primo approccio, che è più semplice e può comunque essere utilizzato in combinazione con il secondo.

#### L'istruzione `include`
L'istruzione `{% include 'nome_template.html' %}` in Jinja consente di inserire l'output renderizzato di un altro template all'interno del template corrente. Questo è utile per evitare la duplicazione di codice e per mantenere il progetto organizzato e modulare.

#### Come usare `include` in Flask

1. **Struttura del progetto**
    Immaginiamo una struttura di progetto Flask come questa:
    ```
    my_flask_app/
    ├── app.py
    ├── templates/
    │   ├── page.html
    │   ├── index.html
    │   ├── partial_header.html
    │   └── partial_footer.html
    ```

2. **Creare template modulari**
    Supponiamo di avere un header e un footer comuni a diverse pagine. Creiamo i file `partial_header.html` e `partial_footer.html` nella cartella `templates`.

    ```html
    <!-- partial_header.html -->
    <header>
        <h1>Benvenuti nel mio sito</h1>
    </header>
    ```

    ```html
    <!-- partial_footer.html -->
    <footer>
        <p>Copyright © 2024</p>
    </footer>
    ```

3. **Utilizzare `include` in un template principale**
    Ora possiamo includere `partial_header.html` e `partial_footer.html` nei nostri template principali come `index.html`.

    ```html
    <!-- index.html -->
    <!DOCTYPE html>
    <html lang="it">
    <head>
        <meta charset="UTF-8">
        <title>Home</title>
    </head>
    <body>
        {% include 'partial_header.html' %}
        
        <main>
            <p>Questo è il contenuto principale della pagina.</p>
        </main>
        
        {% include 'partial_footer.html' %}
    </body>
    </html>
    ```

Stessa cosa si può ora fare con `page.html` e con qualunque altra pagina, mantenentdo l'header e il footer sempre uguali, perché saranno rapresentati sempre dai medesimi file.

#### Perché usare `{% include ... %}`

Utilizzando `{% include ... %}`, possiamo creare applicazioni web Flask più modulari, mantenibili e organizzate.

- **Riutilizzo del codice**: Permette di riutilizzare sezioni comuni come header, footer o sidebar senza dover copiare e incollare lo stesso codice in ogni template.
- **Manutenibilità**: Facilita la manutenzione del codice. Modifiche a header o footer devono essere fatte solo una volta, nel rispettivo file.
- **Organizzazione**: Mantiene i template più puliti e modulari, facilitando la lettura e la gestione del progetto.

## URL, *route* ed *endpoint* in Flask

Se il nostro dominio fosse `mio-sito.com` e avessimo una route come questa:

```py
@app.route('/about')
def about_page():
    return 'Pagina About'
```

- `https://mio-sito.com/about` è l'URL alla pagina.

- `/about` è la *route*, che è un percorso assoluto.

- `about_page` è il nome dell'*endpoint*, che è il nome della funzione che deve gestire la richiesta.

Il nome dell'endpoint è utile per esempio quando si vuole risalire al nome della route che viene gestita da una certa funzione.

Nel nostro esempio, se vogliamo ottenere `/about` sapendo che il nome della funzione è `about_page`, possiamo usare la funzione `url_for()`:

```py
from flask import Flask, url_for

...

@app.route('/about')
def about_page():
    return 'Pagina About'

about_route = url_for('about_page')

print(about_route)  # '/about'
```

## Come leggere i parameri passati con GET e con POST

#### Richiesta GET

I parametri di una richiesta GET sono passati nell'URL. Per esempio: `http://example.com?param1=valore1&param2=valore2`.

Ecco come leggere i parametri in Flask:

```python
from flask import Flask, request

app = Flask(__name__)

@app.route('/get', methods=['GET'])
def get_params():
    param1 = request.args.get('param1')
    param2 = request.args.get('param2')
    return f'Param1: {param1}, Param2: {param2}'

if __name__ == '__main__':
    app.run(debug=True)
```

In questo esempio, `request.args` è un dizionario che contiene i parametri della richiesta GET. Puoi usare `get` per ottenere il valore di un parametro specifico.

### Richiesta POST

I parametri di una richiesta POST sono inviati nel corpo della richiesta stessa e possono essere in formato FormData, JSON, o altri.

#### POST con dati di tipo FormData

Quando inviamo una richiesta POST da un elemento `<form>` tramite il pulsante "submit", la richiesta viene codificata automaticamente in fomato [FormData](https://developer.mozilla.org/en-US/docs/Web/API/FormData).

In questo caso, possiamo usare l'oggetto `request.form`, il quale è un dizionario che contiene i parametri della richiesta POST inviati come FormData:

```python
from flask import Flask, request

app = Flask(__name__)

@app.route('/post', methods=['POST'])
def post_params():
    param1 = request.form.get('param1')
    param2 = request.form.get('param2')
    return f'Param1: {param1}, Param2: {param2}'

if __name__ == '__main__':
    app.run(debug=True)
```

#### POST con dati di tipo JSON

Se i parametri sono in formato JSON, puoi usare invece l'oggetto `request.json`:

```python
@app.route('/post_json', methods=['POST'])
def post_json_params():
    data = request.json
    param1 = data.get('param1')
    param2 = data.get('param2')
    return f'Param1: {param1}, Param2: {param2}'
```

### Riassumendo

Ecco un riassunto di dove possiamo andare a prendere i dati:

- `request.args` per richieste GET
- `request.form` per richieste POST con dati FormData
- `request.json` per richieste POST con dati JSON.

## GET e POST: vantaggi e svantaggi

| **Caratteristica**          | **GET**                                         | **POST**                                        |
|-----------------------------|-------------------------------------------------|-------------------------------------------------|
| **Visibilità dei dati**     | I dati sono visibili nell'URL                  | I dati non sono visibili nell'URL               |
| **Sicurezza**               | Meno sicuro, i dati possono essere salvati nei log del server e nella cronologia del browser | Più sicuro, i dati non vengono salvati nei log del server o nella cronologia del browser |
| **Lunghezza dei dati**      | Limitato dalla lunghezza massima dell'URL (*)       | Non ha limiti specifici sulla lunghezza dei dati |
| **Velocità**                | Generalmente più veloce                        | Generalmente più lento a causa dell'elaborazione aggiuntiva |


In pratica possiamo ricordare:

- **GET**: è comodo, perché possimo salvare richiesta stessa assieme all'URL in modo che, riaprendo quell'URL, anche la richiesta venga inoltrata nuovamente. Ad esempio è molto utile per i motori di ricerca.

- **POST**: è sicuro, perché i dati della richiesta non sono direttamente visibili e possono essere protetti. A esempio è molto utile per l'invio delle credenziali di login a un sito.

(*) NOTA: La lunghezza massima degli URL può variare a seconda del browser e del server utilizzato. Tuttavia, una pratica comune è considerare sicuro un URL che non superi i 2.048 caratteri.

## `if __name__ == '__main__':`

Il pattern `if __name__ == '__main__':` in Python viene utilizzato per verificare se il file di script è in esecuzione come programma principale, ovvero se è stato eseguito come script.

Se la condizione è vera, significa che il file è stato eseguito direttamente e non importato come modulo in un altro script. Questo consente di eseguire un blocco di codice solo quando il file è eseguito autonomamente, ad esempio per testare funzionalità o eseguire script specifici, evitando che lo stesso codice venga eseguito quando il modulo è importato altrove.

Vedi esempio di importazione nel file [mia_altra_applicazione.py](INF_PR_PY_WB_E03/mia_altra_applicazione.py)

## APPROFONDIMENTO Python: `range()` al contrario

In un `range`, se lo `start` è maggiore sello `stop`, allora è necessario indicare uno `step` negativo:

In [ ]:
list(range(10, 3, -1))

[10, 9, 8, 7, 6, 5, 4]